# DESAFÍO SEGUROS

In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

# Preprocesamiento

Cargar la data

In [2]:
df = pd.read_csv('data_desafio_seguros')

Generar variables dummies con one-hot-encoding para las variables descriptivas

In [3]:
df = pd.get_dummies(df, columns = ['0','2','3','3','5','6','8','9','11','13','14','16'])

Booleanizar variables descriptivas o numéricas con sólo dos opciones

In [4]:
df['18'] = df['18'].replace('A191', 0)
df['18']=df['18'].replace('A192',1)
df['19']=df['19'].replace('A201', 0)
df['19']=df['19'].replace('A202', 1)
df['20']=df['20'].replace(2, 0)

Normalizar los datos 

In [ ]:
feature_list = list(df.columns)
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=feature_list)

Separar la data correspondiente a variables (X) y a clasificación (y). 

Dividir en entrenamiento y testing

In [87]:
y = df['20']
X = df.copy()
X = X.drop('20',1)
X = X.drop('row_id',1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Red Neuronal

Se itera sobre el tamaño para minimizar falsos positivos

In [ ]:
max = 400
min_fpr = 1
for i in range (10,max,20):
    for j in range(10,max,20):

        clf = MLPClassifier(activation='relu', hidden_layer_sizes=(i,j), solver='lbfgs')

        neural_model = clf.fit(X_train, y_train)
        y_pred = neural_model.predict(X_test)
        #y_prob= neural_model.predict_proba(X_test)
        tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
        fpr = fp/(tn+fp)
        tpr = tp/(tp+fn)
        if fpr < min_fpr and tpr>fpr:
            min_fpr = fpr
            min_model = clf
        if((i%110==0) or (j%110==0)):
            print(i,j)


Obtener matriz de confusión

In [64]:
y_pred = min_model.predict(X_test)
y_prob= min_model.predict_proba(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
confusion_matrix(y_test,y_pred)

array([[ 49,  36],
       [ 39, 146]], dtype=int64)

Guardar red neuronal 

In [65]:
import pickle 
filename = 'arn.sav'
pickle.dump(min_model, open(filename, 'wb'))


Obtener métricas importantes

In [85]:
fpr = fp/(tn+fp)
tpr = tp/(tp+fn)
ppv = tp/(tp+fp)
print(fpr,tpr,ppv)

0.4235294117647059 0.7891891891891892 0.8021978021978022


In [91]:
y_prob[0]

array([1.22629122e-07, 9.99999877e-01])